In [162]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [163]:
# For understanding the data, mapping words
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [164]:
# Load the pre-trained model
model = tf.keras.models.load_model('imdb_rnn_model_update.keras')

In [165]:
model.summary()  # Check if it's consistent with the training model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,939,077 (15.03 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,052 (10.02 MB)

In [166]:
model.get_weights() 

[array([[-2.0146267e-01, -1.5681446e-01, -1.3160747e-01, ...,
          2.3231667e-01,  1.0393751e+00, -9.8224795e-01],
        [-4.8179384e-02, -6.0766287e-02, -4.1626155e-02, ...,
          3.8805669e-03,  4.1132450e-02, -1.2767346e-03],
        [-7.0045167e-04, -7.7118808e-03,  1.2881058e-01, ...,
          4.9857702e-02,  8.4368050e-02, -5.7094339e-02],
        ...,
        [ 5.1448919e-02,  3.0089995e-02,  7.3076621e-02, ...,
          1.6683979e-02, -8.5547999e-02, -1.3457686e-02],
        [ 4.5791805e-02,  1.1109664e-01,  1.1416423e-01, ...,
         -1.5986361e-01,  2.3693999e-02, -8.1028886e-02],
        [-2.6140444e-02, -1.0015985e-01, -7.3533915e-02, ...,
          1.2387384e-01, -4.7580441e-03,  1.4403093e-01]], dtype=float32),
 array([[ 0.03775856, -0.09899052,  0.04897581, ..., -0.13431191,
          0.09603667, -0.13601123],
        [ 0.15040408,  0.1150291 , -0.03592942, ..., -0.0844624 ,
         -0.06701031,  0.11801624],
        [ 0.01283136,  0.04307311, -0.10171245

In [167]:
# Helper function to decode the reviews
def decode_review(review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in review])

In [168]:
# Helper Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]  # Ensure consistent mapping
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [169]:
# Helper Function to predict the sentiment
def predict_sentiment(review):
    preprocessed_review = preprocess_text(review)
    prediction = model.predict(preprocessed_review)
    print(f"Raw prediction score: {prediction[0][0]}")
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [171]:

# User Input and Prediction
example_review = "The movie was pathetic and boring."
example_review2 = "Bad movie waste of time"
example_review3 = "This movie was fantastic! The acting was great and the plot was thrilling."
sentiment, score = predict_sentiment(example_review3)
print(f'The sentiment of the review is {sentiment} with a score of {score:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Raw prediction score: 0.4641614556312561
The sentiment of the review is Negative with a score of 0.4642
